In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["KAGGLE_USERNAME"] = 'hyunw3'
os.environ["KAGGLE_KEY"] = "b48a329f42b53a6bda2afd4cf341bcf2"
# https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
from sklearn.model_selection import StratifiedKFold

In [3]:
torch.cuda.current_device(), torch.cuda.is_available()

(0, True)

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
train_data = TabularDataset("./train.csv")
train_data = train_data.set_index('id')
subsample_size = 500
train_data.sample(n=subsample_size, random_state=0)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
id,,,,,,,,,,,,,,,,,,,,,
157105,0.225628,0.369954,0.086533,0.220996,0.413357,0.406488,0.502186,0.535051,0.398884,0.211705,...,1,0,0,0,0,0,1,0,0,1
374554,0.156088,0.479218,0.087969,0.335879,0.601985,0.492081,0.361206,0.726592,0.691247,0.483426,...,0,1,1,0,0,0,0,0,0,0
688694,0.246660,0.465889,0.022856,0.310019,0.562853,0.512969,0.393365,0.600628,0.523308,0.205290,...,0,0,0,0,0,0,0,0,0,0
265381,0.253541,0.495103,0.236774,0.215441,0.540664,0.422883,0.409836,0.808747,0.353328,0.537730,...,0,0,0,1,1,0,0,0,0,0
955415,0.289054,0.378237,0.006109,0.272483,0.748737,0.515908,0.362741,0.629370,0.444692,0.211058,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26972,0.245825,0.396460,0.245351,0.232621,0.776057,0.334820,0.817839,0.652080,0.622675,0.143622,...,0,0,0,0,1,0,0,0,0,1
460196,0.161303,0.454262,0.014427,0.260020,0.589798,0.373510,0.475360,0.613104,0.344057,0.142436,...,0,0,0,0,0,1,0,1,1,0
907019,0.229526,0.571115,0.173815,0.508466,0.489185,0.386501,0.465174,0.630219,0.391914,0.470749,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_data = TabularDataset("./test.csv")
test_data = test_data.set_index('id')
subsample_size = 500
test_data.sample(n=subsample_size, random_state=0)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
id,,,,,,,,,,,,,,,,,,,,,
1194257,0.177652,0.402826,0.170229,0.279291,0.449732,0.349957,0.450216,0.676548,0.431982,0.247739,...,1,1,0,0,0,0,1,0,1,0
1396319,0.326258,0.421919,0.162970,0.233764,0.669121,0.363471,0.422830,0.663648,0.598159,0.626254,...,0,0,0,1,0,0,1,0,0,0
1138738,0.207297,0.469348,0.164248,0.335177,0.483002,0.443952,0.358411,0.676115,0.550601,0.268218,...,0,0,0,0,0,0,0,0,0,0
1066570,0.203267,0.562071,0.367516,0.219651,0.656695,0.409316,0.476351,0.657574,0.371572,0.602554,...,0,0,0,0,0,1,1,0,1,0
1149424,0.227488,0.604065,0.090917,0.320712,0.577093,0.367012,0.455212,0.737396,0.531693,0.566927,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176678,0.163133,0.388365,0.007658,0.348349,0.629932,0.330511,0.174350,0.609330,0.555974,0.147360,...,0,0,0,0,0,0,0,0,0,0
1448334,0.161898,0.353998,0.017979,0.245647,0.507021,0.454971,0.506786,0.709880,0.635846,0.147498,...,0,0,1,0,1,0,0,1,0,0
1458895,0.230312,0.390605,0.098201,0.210367,0.582872,0.387836,0.480212,0.589985,0.335449,0.177438,...,1,0,0,1,0,0,0,1,0,0


In [7]:
label='target'
print("summary of class variable : \n",train_data[label].describe())

summary of class variable : 
 count    1000000.000000
mean           0.500485
std            0.500000
min            0.000000
25%            0.000000
50%            1.000000
75%            1.000000
max            1.000000
Name: target, dtype: float64


In [8]:
train_data.shape, test_data.shape

((1000000, 286), (500000, 285))

In [9]:
model_savepath = "./models_ensemble_autogluon/"
#if os.path.exists(model_savepath) is False:
#    os.mkdir(model_savepath)

trainX = train_data.drop(['id','target'],axis=1)
trainY = train_data['target']
testX = test_data.drop(['id'],axis=1)
#testY = test_data.drop(['id'],axis=1)
trainX.shape,trainY.shape


SEEDS = random.choices(range(1,int(1e7)),k=3)
counter = 0
predictor=None
for sidx,seed in enumerate(SEEDS):
    Kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for idx, (train,val) in enumerate(Kfold.split(trainX,trainY)):
        train_x, train_y = trainX.iloc[train], trainY.iloc[train]
        val_x, val_y = trainX.iloc[val], trainY.iloc[val]
        if counter == 0:
            predictor = TabularPredictor(label=label, path=model_savepath).fit([train_x,train_y],[val_x,val_y],
                                                                  presets='best_quality',auto_stack=True)
        else :
            predictor = TabularPredictor(label=label, eval_metric=['roc_auc'], path=model_savepath).fit_Extra([train_x,train_y],[val_x,val_y],
                                                        time_limit =30000 ,presets='best_quality',auto_stack=True)
            
    break

In [10]:
time_limit = 5*60*60
predictor = TabularPredictor( label=label,eval_metric='roc_auc', path=model_savepath).fit(train_data, ag_args_fit={'num_gpus': 1},
                                                                  num_bag_folds=5, num_bag_sets =1, num_stack_levels=2,
                                                                                            presets='best_quality')

Presets specified: ['best_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./models_ensemble_autogluon/"
AutoGluon Version:  0.3.2b20211005
Train Data Rows:    1000000
Train Data Columns: 285
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    528783.49 MB
	Train Data (Original)  Mem

KeyboardInterrupt: 

In [ ]:
y_pred = predictor.predict(test_data)
y_pred

In [ ]:
test_pred = pd.DataFrame({'id' : y_pred.index.values,'target' :y_pred})
test_pred.to_csv("./submit_1014_1.csv",index=False)
test_pred.head()

In [ ]:
d_frame = pd.DataFrame()
d_frame["id"] = y_pred.index
d_frame["target"] = y_pred
d_frame

In [ ]:
d_frame

In [ ]:
d_frame.drop([""])

In [ ]:
predictor.leaderboard(train_data, silent=True)

In [ ]:
results = predictor.fit_summary(show_plot=True)
results